### 1. Load data

Code written for you already.

---

In [21]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import cross_val_score

from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNetCV
from sklearn.linear_model import Lasso, Ridge, ElasticNet

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor

%matplotlib inline

In [22]:
red = pd.read_csv('./datasets/winequality-red.csv', sep=';')
white = pd.read_csv('./datasets/winequality-white.csv', sep=';')

In [13]:
print red.shape, white.shape

(1599, 12) (4898, 12)


In [14]:
print red.columns
print white.columns

Index([u'fixed acidity', u'volatile acidity', u'citric acid',
       u'residual sugar', u'chlorides', u'free sulfur dioxide',
       u'total sulfur dioxide', u'density', u'pH', u'sulphates', u'alcohol',
       u'quality'],
      dtype='object')
Index([u'fixed acidity', u'volatile acidity', u'citric acid',
       u'residual sugar', u'chlorides', u'free sulfur dioxide',
       u'total sulfur dioxide', u'density', u'pH', u'sulphates', u'alcohol',
       u'quality'],
      dtype='object')


In [15]:
red['is_red'] = 1
white['is_red'] = 0
wine = pd.concat([red, white], axis=0)

In [17]:
wine.dropna(inplace=True)

In [18]:
print wine.shape
wine.head(3)

(6497, 13)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,is_red
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,1


### 2. Create X, Y, and normalize X

**X should be quality**. You are going to do regression rather than classification on the wine this time.

---

### 3. Linear regression

1. Cross-validate a linear regression with 5 folds. 
2. Build a linear regression model with the full X and Y.

---

### 4. Ridge regression

1. Either use `GridSearchCV` or `RidgeCV` to find the best `C` or `alpha` respectively. **Remember that bigger alphas means stronger regularization, and smaller Cs are stronger regularization. (C is the inverse of alpha).**
2. Cross-validate the R2 with Ridge using your optimal C or alpha.
3. Build a final Ridge model and fit it on the full X and Y as you did above.

---

### 5. Lasso regression

1. Use either `GridSearchCV` or `LassoCV` to find the optimal `C` or `alpha` for the Lasso regression.
2. Cross-validate the R2 with Lasso using your optimal C or alpha.
3. Build a final Lasso model fit on the full X and Y.

---

### 6. ElasticNet regression

Now you'll get to try out the ElasticNet. It is a combination of the Ridge and the Lasso to leverage the benefits of both!

Arguments to optimize:

    alpha : same as the Ridge/Lasso above
    l1_ratio: this is the proportion of Ridge vs Lasso that the model is. 
        An l1_ratio of 0.0 is a pure Ridge
        An l1_ratio of 1.0 is a pure Lasso
        
1. Use `GridSearchCV` or `ElasticNetCV` to search for the optimal `alpha` and `l1_ratio`. 
2. Explain the probable reason why the it chose the parameters it did as the best ones.
3. Cross-validate the R2 with the ElasticNet using the optimal parameters.
4. Fit the ElasticNet on all X and Y.

---

### 7. DecisionTreeRegressor

1. Use `GridSearchCV` to find the best `max_features`, `max_depth`, and `min_samples_leaf`. Read the documentation and think about what range of parameters would be good to search for each!
2. Cross-validate the R2 as above.
3. Fit a DecisionTreeRegressor on all X and Y as above.

---

### 8. BaggingRegressor

Now we'll use bagging with the DecisionTreeRegressor. Yes, regressions can be done with bagging too!

---

Remember that with the Bagging regressor you first have to initialize the internal "base estimator" that it will copy:

```python
dtr = DecisionTreeRegressor()
```

A cool thing to note is that you can actually gridseach over the internal base estimators as well. So, not only are you finding the best parameters for the BaggingRegressor but also the DecisionTreeRegressors that it copies inside:

```python
bag_params = {
    'base_estimator__max_features':[None],
    'base_estimator__max_depth':[None],
    'base_estimator__min_samples_leaf':[1],
    'max_features': [0.33, 0.66, 0.99],
    'max_samples': [0.1, 0.2, 0.4, 0.6, 0.8, 0.9],
    'n_estimators': [100]
}
```

**Be careful putting too many parameters into the `GridSearchCV`! It can really explode the possible permutations!**

That being said, you'll probably be able to put a decent amount of parameters in since the wine dataset doesn't have many columns.

Next you initialize the BaggingRegressor, putting the desired model as the first argument:

```python
bag = BaggingRegressor(dtr)
```

This tells the BaggingRegressor that you want it to spawn DecisionTreeRegressors as it's internal "children" base estimators.

Lastly, you'll put the BaggingClassifier into the grid searcher and fit on the data (it will cross-validate with the specified `cv` folds.

```python
bag_gs = GridSearchCV(bag, bag_params, cv=5, verbose=1)
bag_gs.fit(X, Y)
```

---

As before...

1. Use `GridSearchCV` to find the best `BaggingRegressor` and optionally internal `DecisionTreeRegressor` parameters.
2. Cross-validate the R2.
3. Fit a `BaggingRegressor` on all X and Y with the optimal parameters.

---

### 9. Get feature importances from `RandomForestRegressor`

The `RandomForestRegressor` has an attribute called `.feature_importances_`. These are the importances of your predictors as measured by how useful they were to the base estimators.

As you may recall, the `RandomForestRegressor` is just a special case of the `BaggingRegressor` that specifically uses decision trees. In fact, you've already done it above. The difference is that this class gives us the feature importances whereas the "generalized" bagging regressor class does not.

1. Save the column names X in a variable.
2. Grid search optimal parameters for the `RandomForestRegressor`.
3. Fit a `RandomForestRegressor` using the optimal parameters you found on the full X and Y.
4. Get out the feature importances.
5. Create a pandas DataFrame where one column is the feature importances and the other column is the X column names.
6. Sort the dataframe you made by feature importances in descending order.
7. Plot the feature importances.

---

### 10. [BONUS] Use a different regression class with the `BaggingRegressor`

You could try `Ridge`, `Lasso`, `ElasticNet`, `SVC`, `KNeighborsRegressor` or any kind of regression class you're interested in!

---